In [2]:
import json
import altair as alt
from altair import expr, datum
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests

In [130]:
LOCAL=False

if LOCAL: local_suffix='_local'
else: local_suffix=''

In [131]:
%%capture pwd
!pwd

In [132]:
# uid = "2021-05-05-which-firms-and-industries-have-been-most-affected-by-covid-update"  # article unique ID
uid = pwd.stdout.split('/')[-1].split('\r')[0]
eco_git_home = "https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/"
eco_git_path = (
    eco_git_home+"articles/"
    + uid
    + "/data/"
)
vega_embed = requests.get(eco_git_home+'guidelines/html/vega-embed.html').text
colors = json.loads(requests.get(eco_git_home+"guidelines/colors/eco-colors.json").content)
category_color = json.loads(
    requests.get(eco_git_home+"guidelines/colors/eco-category-color.json").content
)
hue_color = json.loads(
    requests.get(eco_git_home+"guidelines/colors/eco-single-hue-color.json").content
)
mhue_color = json.loads(
    requests.get(eco_git_home+"guidelines/colors/eco-multi-hue-color.json").content
)
div_color = json.loads(
    requests.get(eco_git_home+"guidelines/colors/eco-diverging-color.json").content
)
config = json.loads( requests.get(eco_git_home+"guidelines/charts/eco-global-config.json").content)
height = config["height"]
width = config["width"]
height, width

(300, 500)

# Fig 1

In [133]:
df = (
    pd.read_excel("raw/Econ Observatory Henley chart data.xlsx", 
                  sheet_name="Sheet1", skiprows=23)
    .dropna(how="all", axis=1)
    .dropna(how="all", axis=0)
)
df=df.set_index('Date').stack().reset_index()
df.columns=['date','region','value']

In [134]:
f = "fig1_GDP_country"
f1 = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f1.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL: f1=df
df.head()

,date,region,value
0,2018Q2,UK,0.3
1,2018Q2,England,0.3
2,2018Q2,Wales,0.8
3,2018Q2,Scotland,0.6
4,2018Q2,Northern Ireland,0.9


In [135]:
bars = (
    alt.Chart(f1)
    .encode(
        y=alt.Y('value:Q',stack=False,title='',axis=alt.Axis(
            grid=True,title='%',
            titleAnchor='start',labelColor=colors['eco-gray'],titleColor=colors['eco-gray'],
            tickColor=colors['eco-gray'],domainColor=colors['eco-gray'],
            titleFontSize=10,titleFontWeight="normal",
            ticks=False,
            labelAlign="left",
            labelBaseline="middle",
            labelPadding=-5,
            labelOffset=-10,
            titleX=25,
            titleY=295,
            titleBaseline='bottom',
            titleAngle=0,
            titleAlign="left",
            tickCount=4,
            format=".0f",
        )),
        x=alt.X('date:N',title='',axis=alt.Axis(labelColor=colors['eco-gray'],titleColor=colors['eco-gray'],
            tickColor=colors['eco-gray'],domainColor=colors['eco-gray'],orient='top',
            labelExpr="indexof(datum.value,'Q1')>-1?slice(datum.value,0,4)+' '+slice(datum.value,4):slice(datum.value,4)",labelAngle=0,
            ))
    )
)

bars1=bars.mark_bar(size=8,xOffset=-12,stroke=colors['eco-gray'],color='white',opacity=0.8)\
    .transform_filter("datum.region=='England'")
bars2=bars.mark_bar(size=8,xOffset=-4,color=colors['eco-pink'],opacity=0.8)\
    .transform_filter("datum.region=='Wales'")
bars3=bars.mark_bar(size=8,xOffset=4,color=colors['eco-mid-blue'],opacity=0.8)\
    .transform_filter("datum.region=='Scotland'")
bars4=bars.mark_bar(size=8,xOffset=12,color=colors['eco-green'],opacity=0.8)\
    .transform_filter("datum.region=='Northern Ireland'")
line=bars.mark_line(color=colors['eco-blue'],opacity=0.8)\
    .transform_filter("datum.region=='UK'")

text=line.mark_text(yOffset=15,xOffset=-5,size=10,align='right',color=colors['eco-blue']).encode(
    text=alt.Text('value:Q',format='.1f')
)

layer = (
    bars1+bars2+bars3+bars4+line+text
).configure_view(stroke=None).properties(title="").properties(height=300, width=400)
layer.save("visualisation/" + f + ".json")
layer

alt.LayerChart(...)